In [2]:
import pandas as pd
import sqlite3
import csv
import seaborn as sns

Creating a connection and database (Regular SQL method)

In [3]:
conn = sqlite3.connect('Databankdraft')
c = conn.cursor()

Importing and loading datasets with pandas

In [4]:
df_sales = pd.read_csv ('Sales.csv')
df_menu = pd.read_csv ('menu.csv')
df_mem = pd.read_csv ('members.csv')

Clean space in column headers

In [5]:
df_sales.columns

Index(['   customer_id', ' order_date', ' product_id'], dtype='object')

In [6]:
df_sales.columns = df_sales.columns.str.strip()

In [7]:
df_sales.columns

Index(['customer_id', 'order_date', 'product_id'], dtype='object')

In [8]:
df_menu.columns

Index(['  product_id', ' product_name', ' price'], dtype='object')

In [9]:
df_menu.columns = df_menu.columns.str.strip()

In [10]:
df_menu.columns

Index(['product_id', 'product_name', 'price'], dtype='object')

In [11]:
df_mem.columns = df_mem.columns.str.strip()

In [12]:
df_mem.columns

Index(['customer_id', 'join_date'], dtype='object')

Inserting values into a table

In [13]:
df_sales.to_sql('df_sales', conn,if_exists='replace',index=False)
df_menu.to_sql('df_menu', conn,if_exists='replace',index=False)
df_mem.to_sql('df_mem', conn,if_exists='replace',index=False)

Verifying Data types

In [14]:
qry = """PRAGMA table_info(df_sales)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,0
1,1,order_date,TEXT,0,None,0
2,2,product_id,INTEGER,0,None,0


In [15]:
qry = """PRAGMA table_info(df_menu)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,product_id,INTEGER,0,None,0
1,1,product_name,TEXT,0,None,0
2,2,price,INTEGER,0,None,0


In [16]:
qry = """PRAGMA table_info(df_mem)"""
pd.read_sql_query(qry, con=conn)

,cid,name,type,notnull,dflt_value,pk
0,0,customer_id,TEXT,0,None,0
1,1,join_date,TEXT,0,None,0


In [17]:
qry = """SELECT
  	product_id,
   product_name,
   price
FROM df_menu
ORDER BY price DESC;"""

pd.read_sql_query(qry, con=conn)

,product_id,product_name,price
0,2,curry,15
1,3,ramen,12
2,1,sushi,10


What is the total amount each customer spent at the restaurant?

In [48]:
qry = """SELECT df_sales.customer_id,
SUM(price)
FROM df_sales
INNER JOIN df_menu ON df_sales.product_id = df_menu.product_id GROUP BY df_sales.customer_id;
"""

pd.read_sql_query(qry, con=conn)

,customer_id,SUM(price)
0,A,76
1,B,74
2,C,36


How many days has each customer visited the restaurant?

In [53]:
qry = """ SELECT customer_id,
COUNT (order_date)
from df_sales GROUP BY customer_id;
"""

pd.read_sql_query(qry, con=conn)

,customer_id,COUNT (order_date)
0,A,6
1,B,6
2,C,3


What was the first item from the menu purchased by each customer?

In [85]:
qry = """ SELECT df_menu.product_name, df_sales.order_date, df_sales.customer_id
from df_sales
INNER JOIN df_menu ON df_sales.product_id = df_menu.product_id GROUP BY df_sales.customer_id;
"""

pd.read_sql_query(qry, con=conn)

,product_name,order_date,customer_id
0,sushi,2021-01-01,A
1,curry,2021-01-01,B
2,ramen,2021-01-01,C


What is the most purchased item on the menu and how many times was it purchased by all customers?

In [103]:
qry = """ SELECT MAX(df_menu.product_name),
COUNT(df_sales.product_id)
from df_sales
INNER JOIN df_menu ON df_sales.product_id = df_menu.product_id GROUP BY df_sales.product_id
ORDER BY df_sales.product_id DESC
LIMIT 1;
"""

pd.read_sql_query(qry, con=conn)

,MAX(df_menu.product_name),COUNT(df_sales.product_id)
0,ramen,8


Which item was the most popular for each customer?